# FIT3182 - Big data management and processing

Name: Cheong Karr Kei

Student ID: 30091497

Email: kche0070@student.monash.edu


# Assignment Part B #

**Task 1. Processing Stream Data**

**Event Producer 2**

In this notebook, we have Event Producer 2 which loads data from `hotspot_AQUA_streaming.csv`. This program acts as one of the producers in our stream and feeds fire data to our stream every 2 seconds under the topic `FIT3182_Assignment`. The key for this producer is set as `P2`.



In [ ]:
# import statements
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import datetime as dt
import csv
from datetime import timedelta

 
def publish_message(producer_instance, topic_name, key, data):
    try:
        #First, enconde the value or key into bytes
        key_bytes = bytes(key, encoding='utf-8')
        
        #Publish data 
        producer_instance.send(topic_name, key=key_bytes, value=data)
        print('Message published successfully. ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
    
    #Set topic
    topic = 'FIT3182_Assignment'
    #Set key as P2 for Producer 2
    key = "P2"
    print('Publishing records..')
    producer01 = connect_kafka_producer()
    
    #initialise counter 
    counter = 1
    
    #array to store fire  data 
    fire_data = []
    
    #open csv file containing fire data by AQUA

    header = None
    with open('hotspot_AQUA_streaming.csv', newline='') as f:
        reader = csv.reader(f)
        #get the header (first row)
        header = next(reader)  

        #iterate row 2 onwards
        for row in reader:


            #create new document
            newFire = {
                            #latitude
                            header[0] : float(row[0]),    
                            #longitude
                            header[1]: float(row[1]),   
                            #confidence
                            header[2]: int(row[2]),
                            #surface_temperature_celcius
                            header[3]: int(row[3]),
                            
                          } 
            
            #add to array
            fire_data.append(newFire)
            
    #send data while list of fire data is still empty
    while len(fire_data)>0:
        index = random.randrange(len(fire_data))
        data = fire_data.pop(index)
            
        #simulate datetime 
        current_dt = dt.datetime.now() + timedelta(hours=counter)
        current_dt = current_dt.replace(microsecond=0).isoformat('T')

        #add datetime
        data["datetime"] = current_dt

        #send document 
        publish_message(producer01, topic, key, data)

        #increment counter 
        if counter <= 24:
            counter += 1
        else:
            counter = 1

        #wait 2 seconds 
        sleep(2)

    
